## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import random
import pandas as pd
import gmaps
import numpy as np
import timeit
from scipy.stats import linregress 
import matplotlib.pyplot as plt
from config import g_key
gmaps.configure(api_key=g_key)
from citipy import citipy
import requests
from datetime import datetime
import time
from scipy.stats import linregress 
from config import weather_api_key

In [14]:
#Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,82.40,69,20,8.05,few clouds
1,1,Santo Tomas,PH,14.1079,121.1414,89.01,66,75,4.61,broken clouds
2,2,Wilmington,US,34.2257,-77.9447,73.00,78,1,17.27,clear sky
3,3,Albany,US,42.6001,-73.9662,60.80,54,75,1.19,broken clouds
4,4,Dingle,PH,10.9995,122.6711,87.10,62,85,7.72,overcast clouds


In [16]:
#Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [17]:
#Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preffered_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,82.40,69,20,8.05,few clouds
1,1,Santo Tomas,PH,14.1079,121.1414,89.01,66,75,4.61,broken clouds
4,4,Dingle,PH,10.9995,122.6711,87.10,62,85,7.72,overcast clouds
12,12,Neiafu,TO,-18.6500,-173.9833,82.40,69,40,4.61,scattered clouds
16,16,Butaritari,KI,3.0707,172.7902,82.13,75,54,7.02,light rain
...,...,...,...,...,...,...,...,...,...,...
640,640,Upata,VE,8.0086,-62.3989,82.00,77,91,1.99,overcast clouds
645,645,Port-Gentil,GA,-0.7193,8.7815,81.18,78,100,3.02,overcast clouds
653,653,Ahuimanu,US,21.4447,-157.8378,80.60,57,20,12.66,few clouds
656,656,Buala,SB,-8.1450,159.5921,84.58,69,49,2.53,light rain


In [18]:
#Determine if there are any empty rows.
preffered_cities_df.count()

City_ID                115
City                   115
Country                115
Lat                    115
Lng                    115
Max Temp               115
Humidity               115
Cloudiness             115
Wind Speed             115
Current Description    115
dtype: int64

In [19]:
#Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preffered_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,82.40,69,20,8.05,few clouds
1,1,Santo Tomas,PH,14.1079,121.1414,89.01,66,75,4.61,broken clouds
4,4,Dingle,PH,10.9995,122.6711,87.10,62,85,7.72,overcast clouds
12,12,Neiafu,TO,-18.6500,-173.9833,82.40,69,40,4.61,scattered clouds
16,16,Butaritari,KI,3.0707,172.7902,82.13,75,54,7.02,light rain
...,...,...,...,...,...,...,...,...,...,...
640,640,Upata,VE,8.0086,-62.3989,82.00,77,91,1.99,overcast clouds
645,645,Port-Gentil,GA,-0.7193,8.7815,81.18,78,100,3.02,overcast clouds
653,653,Ahuimanu,US,21.4447,-157.8378,80.60,57,20,12.66,few clouds
656,656,Buala,SB,-8.1450,159.5921,84.58,69,49,2.53,light rain


In [20]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

#Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,82.40,few clouds,-21.2000,-175.2000,
1,Santo Tomas,PH,89.01,broken clouds,14.1079,121.1414,
4,Dingle,PH,87.10,overcast clouds,10.9995,122.6711,
12,Neiafu,TO,82.40,scattered clouds,-18.6500,-173.9833,
16,Butaritari,KI,82.13,light rain,3.0707,172.7902,
18,Marawi,PH,82.99,overcast clouds,7.9986,124.2928,
23,Kuala Terengganu,MY,80.60,few clouds,5.3302,103.1408,
27,Honiara,SB,83.86,broken clouds,-9.4333,159.9500,
31,Georgetown,MY,86.00,few clouds,5.4112,100.3354,
33,Cabo San Lucas,MX,84.20,clear sky,22.8909,-109.9124,


In [21]:
#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
#Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found...skipping') 
hotel_df.head()

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,82.40,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Santo Tomas,PH,89.01,broken clouds,14.1079,121.1414,Microtel by Wyndham Batangas
4,Dingle,PH,87.10,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Neiafu,TO,82.40,scattered clouds,-18.6500,-173.9833,Mystic Sands
16,Butaritari,KI,82.13,light rain,3.0707,172.7902,Isles Sunset Lodge


In [27]:
#Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,82.40,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Santo Tomas,PH,89.01,broken clouds,14.1079,121.1414,Microtel by Wyndham Batangas
4,Dingle,PH,87.10,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Neiafu,TO,82.40,scattered clouds,-18.6500,-173.9833,Mystic Sands
16,Butaritari,KI,82.13,light rain,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
640,Upata,VE,82.00,overcast clouds,8.0086,-62.3989,Posada Villa Nela
645,Port-Gentil,GA,81.18,overcast clouds,-0.7193,8.7815,Fengshui Residence
653,Ahuimanu,US,80.60,few clouds,21.4447,-157.8378,Paradise Bay Resort
656,Buala,SB,84.58,light rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [32]:
#Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacations"
#Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
#Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

#Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))